# PyTorch: Variable, Gradientes e Grafo Computacional

## Atualizando para usar Tensores com `requires_grad=True`

A classe `Variable` foi descontinuada no PyTorch. Agora, a funcionalidade de rastreamento de gradiente está integrada diretamente aos tensores. Para habilitar o cálculo automático de gradientes para um tensor, defina o parâmetro `requires_grad` como `True` durante a criação do tensor ou use o método `.requires_grad_(True)`.

## Objetivos

Este notebook introduz
- o conceito de `Variables` do PyTorch,
- uma interpretação numérica intuitiva do gradiente, e o
- grafo computacional, utilizado para o cálculo automático do gradiente de uma função.

Um dos principais fundamentos para que o PyTorch seja adequado para deep learning é a sua habilidade de
calcular o gradiente automaticamente a partir da expressões definidas. Essa facilidade é implementada
pelo tipo Variable do PyTorch, que adiciona ao tensor a facilidade de cálculo automático do gradiente pela construção dinâmica do grafo computacional.

## Grafo computacional

```
    y_pred = x * w
    e = y_pred - y
    e2 = e**2
    J = e2.sum()
```

![alt text](https://raw.githubusercontent.com/vcasadei/images/master/GrafoComputacional.png)

Variable possui 3 campos: o dado em si (data), o gradiente (grad) e um apontador (creator) para construir o grafo da backpropagation. Uma expressão utilizada para o cálculo do gradiente exige que todas suas expressões sejam calculadas com Variables, caso contrário não é possível construir o grafo computacional.

![alt text](https://raw.githubusercontent.com/vcasadei/images/master/variables.png)

In [1]:
import torch
# from torch.autograd import Variable # Variable is deprecated

## Variable é criada a partir de um tensor e possui as mesmas funcionalidades

In [2]:
y_t = 2 * torch.arange(0.,4.)
y = y_t; y # Use tensor directly

tensor([0., 2., 4., 6.])

In [3]:
x = torch.arange(0.,4.); x # Use tensor directly

tensor([0., 1., 2., 3.])

In [4]:
w = torch.ones(1, requires_grad=True); w # Use tensor with requires_grad=True

tensor([1.], requires_grad=True)

## Cálculo automático do gradiente da função perda J

Seja a expressão: $$ J = ((x  w) - y)^2 $$

Queremos calcular a derivada de $J$ em relação a $w$.

### Montagem do grafo computacional

In [5]:
# predict (forward)
y_pred = x * w

# cálculo da perda J: loss
e = y_pred - y
e2 = e.pow(2)
J = e2.sum()
J

tensor(14., grad_fn=<SumBackward0>)

## Auto grad - processa o grafo computacional backwards

O `backward()` varre o grafo computacional a partir da variável a ele associada e calcula o gradiente para todas as `Variables` que possuem o atributo `requires_grad` como verdadeiro.
O `backward()` destroi o grafo após sua execução. Isso é intrínsico ao PyTorch pelo fato dele ser uma rede dinâmica.

In [6]:
J.backward()
print(w.grad)

tensor([-28.])


In [7]:
w.grad.data.zero_();

## Interpretação do Gradiente

O gradiente de uma variável final (J) com respeito à outra variável de entrada (w) pode ser interpretado como o quanto a variável final J vai aumentar se houver um pequeno aumento na variável de entrada (w).
Por exemplo suponha que o gradiente seja 28. Isto significa se aumentarmos a variável w de 0.001, então J vai aumentar de 0.028.

In [8]:
eps = 0.001
y_pred = x * (w+eps)
J_new = (y_pred - y).pow(2).sum()
J_new

tensor(13.9720, grad_fn=<SumBackward0>)

In [9]:
print((J_new - J).data.numpy())

-0.027988434


## Back propagation

Uma forma equivalente explícita de calcular o gradiente é fazendo o processamento do backpropagation no grafo computacional, de forma explícita.
Apenas como ilustração.

Função de perda:
$$ J(\hat{y_i},y_i) = \frac{1}{M} \sum_{i=0}^{M-1} (\hat{y_i} - y_i)^2 $$

Gradiente:
$$  \mathbf{\nabla{J_w}} = \frac{2}{M}\mathbf{x^T}(\mathbf{x w^T} - \mathbf{y}) $$

Atualização dos parâmetros pelo gradiente descendente:
$$ \mathbf{w} = \mathbf{w} − \eta (\mathbf{\nabla J_w})^T $$

In [10]:
import numpy as np

dJ = 1.
de2 = dJ * np.ones((4,))
de = de2 * 2 * e.data.numpy()
dy_pred = de
dw = (dy_pred * x.data.numpy()).sum()
print(dJ)
print(de2)
print(de)
print(dw)

1.0
[1. 1. 1. 1.]
[ 0. -2. -4. -6.]
-28.0


# Exercícios

## Questões

1. O que acontece com o grafo computacional após execução do `backward()`?
>  Após executar o PyTorch percorre todo o grafo computacional de trás para frente, acumulando em cada tensor. Depois de calcular todos os gradientes, o PyTorch descarta o grafo, liberando a memória.

## Atividades

1. Execute um passo de atualização do valor de w, pelo
gradiente descendente. Utilize um fator de aprendizado (*learning rate*) de 0.1
para atualizar o `w`. Após, recalcule a função de perda:

    - w = w - lr * w.grad.data
    - execute a célula 1.3.1 e verifique o quanto que a perda J diminuiu
    

In [11]:
# Passo 1: atualizar w
lr = 0.1
w.data = w.data - lr * w.grad.data

# Passo 2: zerar gradiente
w.grad.data.zero_()

# Passo 3: recalcular J
y_pred = x * w
J = (y_pred - y).pow(2).sum()
print("Nova Loss J:", J.item())

Nova Loss J: 14.0


# Aprendizados com este notebook

1. Tensors com requires_grad=True

O PyTorch utiliza tensores que armazenam não apenas valores numéricos, mas também gradientes.

Quando requires_grad=True, o PyTorch registra as operações para construir um grafo computacional.

2. Grafo computacional

Cada operação matemática cria nós no grafo.

Esse grafo representa a cadeia de operações necessárias para calcular o gradiente da função final.

Após chamar backward(), o grafo é destruído para economizar memória.

3. Gradiente numérico

Utilizando pequenas perturbações ε é possível aproximar o gradiente numericamente:

𝑓( 𝑤 + 𝜀)
−
𝑓
(
𝑤
)
𝜀
ε
f(w+ε)−f(w)
	​


Isso serve para verificar se o gradiente analítico/autograd está correto.

4. Autograd e backward()

O PyTorch calcula automaticamente os gradientes usando backpropagation.

O gradiente aparece em w.grad.

Os gradientes são acumulados, por isso devem ser zerados antes de novo passo.

5. Gradiente descendente manual

A atualização de pesos pode ser feita manualmente:

𝑤
:
=
𝑤
−
𝜂
⋅
∂
𝐽
∂
𝑤
w:=w−η⋅
∂w
∂J
	​


Isso permite aprender como frameworks funcionam internamente, antes de usar otimizadores mais avançados.